In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define constants
IMAGE_SIZE = 150
BATCH_SIZE = 32

# Set up data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
    'Tomatoes/Train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['good', 'bad']
)

test_generator = test_datagen.flow_from_directory(
    'Tomatoes/Test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['good', 'bad']  # Explicitly specify class labels
)

Found 56 images belonging to 2 classes.
Found 69 images belonging to 2 classes.


In [4]:

# Create the CNN model
model = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ],
    name='tomato_classifier'
)

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Train the model using the generators
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6338 - accuracy: 0.6875 - val_loss: 2.8359 - val_accuracy: 0.4844
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 1.0758 - accuracy: 0.7500 - val_loss: 0.4609 - val_accuracy: 0.8281
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 0.4218 - accuracy: 0.7500 - val_loss: 0.2948 - val_accuracy: 0.8125
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 0.1245 - accuracy: 1.0000 - val_loss: 0.3038 - val_accuracy: 0.7500
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 0.0618 - accuracy: 1.0000 - val_loss: 0.4713 - val_accuracy: 0.7344
Epoch 6/10
1/1 [==============================] - 1s 981ms/step - loss: 0.0864 - accuracy: 0.9583 - val_loss: 0.2166 - val_accuracy: 0.8281
Epoch 7/10
1/1 [==============================] - 1s 927ms/step - loss: 0.0407 - accuracy: 0.9583 - val_loss: 0.0810 - val_accuracy: 1.0000
Epoch 8/10
1/1 [===================

In [7]:
# Save the model
model.save('tomato_classifier_model.h5')

C:\Users\brigh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from tensorflow.keras.applications import VGG16

# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Freeze all the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained base model
model2 = Sequential([
    vgg16_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [13]:
# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model using the generators
model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.9252 - accuracy: 0.3750 - val_loss: 1.5498 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 0.6847 - accuracy: 0.7500 - val_loss: 0.8771 - val_accuracy: 0.6094
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 0.3082 - accuracy: 0.8125 - val_loss: 0.1977 - val_accuracy: 0.9062
Epoch 4/10
1/1 [==============================] - 6s 6s/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 0.1021 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 6s 6s/step - loss: 0.1018 - accuracy: 1.0000 - val_loss: 0.1086 - val_accuracy: 0.9531
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 0.0737 - accuracy: 1.0000 - val_loss: 0.0988 - val_accuracy: 0.9531
Epoch 7/10
1/1 [==============================] - 6s 6s/step - loss: 0.0306 - accuracy: 1.0000 - val_loss: 0.0715 - val_accuracy: 0.9688
Epoch 8/10
1/1 [=========================

In [15]:
# Save the model
model2.save('tomato_classifier_VGG16.h5')

C:\Users\brigh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Assuming you've trained and saved your model already...

# Load the saved models
model = tf.keras.models.load_model('tomato_classifier_model.h5')
loaded_model = tf.keras.models.load_model('tomato_classifier_VGG16.h5')

# Now, assuming you have a new image you want to classify
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the new image
new_image_path = 'Tomatoes/Test/bad/116_100.jpg'  # Replace with the path to your new image
new_img = image.load_img(new_image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0)
new_img_array /= 255.  # Normalize the image

# Make a prediction using the loaded model
prediction = model.predict(new_img_array)
prediction1 = loaded_model.predict(new_img_array)

# Interpret the prediction
print("prediction of model 1")
if prediction[0] < 0.5:
    print("The tomato is classified as 'good'.")
else:
    print("The tomato is classified as 'bad'.")

print("prediction of model vgg16")
if prediction1[0] < 0.5:
    print("The tomato is classified as 'good'.")
else:
    print("The tomato is classified as 'bad'.")

1/1 [==============================] - 0s 229ms/step
prediction of model 1
The tomato is classified as 'bad'.
prediction of model vgg16
The tomato is classified as 'bad'.


In [27]:
import pandas as pd
import os
import csv
import base64
import numpy as np
from PIL import Image

# Define constants
IMAGE_SIZE = 150  # Adjust this according to your image size
dataset_path = 'Tomatoes'  # Replace this with your dataset path
csv_file_path = 'tomato_dataset_with_images.csv'
excel_file_path = 'tomato_dataset.xlsx'

# Open CSV file in write mode
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Label', 'Image', 'Set'])  # Write header

    # Iterate through Train and Test folders
    for folder in ['Train', 'Test']:
        for class_name in ['good', 'bad']:
            class_dir = os.path.join(dataset_path, folder, class_name)
            for filename in os.listdir(class_dir):
                file_path = os.path.join(class_dir, filename)

                # Check if the file is an image by attempting to open it with PIL
                try:
                    img = Image.open(file_path)
                    img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
                except (IOError, OSError):
                    continue  # Skip if it's not an image file

                # Convert image to numpy array
                img_array = np.array(img)
                
                # Convert image array to base64 string
                img_str = img_array.tobytes()
                img_base64 = base64.b64encode(img_str).decode('utf-8')

                # Determine if the image is in the 'Train' or 'Test' set
                image_set = folder

                writer.writerow([file_path, class_name, img_base64, image_set])

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)

# Save the DataFrame to an Excel file
data.to_excel(excel_file_path, index=False)
